In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [41]:
# Create imputer object
imputer = IterativeImputer(max_iter=100, tol=1e-3)

df = pd.read_csv('data/iith_foml_2023_train.csv')
y_train = df.iloc[:, -1]
X_train = df.iloc[:, :-1]
# Fit the imputer model on the dataset to learn the data patterns
# X_train.drop(columns=['Feature 16', 'Feature 17'], inplace=True)
imputer.fit(X_train)

# Transform the dataset to replace missing values
# Convert back to a DataFrame
X_train_imputed = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)

In [42]:
df_test = pd.read_csv('data/iith_foml_2023_test.csv')
# df_test.drop(columns=['Feature 16', 'Feature 17'], inplace=True)
X_test = pd.DataFrame(imputer.transform(df_test), columns=df_test.columns)

In [45]:
from sklearn.neural_network import MLPClassifier

# Initialize mlpoost model
mlp_model = MLPClassifier(hidden_layer_sizes=[16, 16], max_iter=5000, activation='logistic')

# Train the model on the training data
mlp_model.fit(X_train_imputed, y_train)

# Make predictions on the validation data
mlp_predictions = mlp_model.predict(X_test)

In [46]:
pred_df = pd.DataFrame()
pred_df['Category'] = mlp_predictions
pred_df['id'] = pred_df.index + 1
pred_df = pred_df[['id', 'Category']]
pred_df.to_csv('output/mlp.csv', index=False)